In [18]:
import os
import psycopg2
from psycopg2 import sql

os.environ["PGDATABASE"] = "ernieplus"

def select_rows_with_range(start_index, end_index):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")  # Replace with your connection parameters
        cur = conn.cursor()

        # Execute the SELECT query to obtain rows within the specified range
        cur.execute("SELECT * FROM hm31.in_edges_features_cert  ORDER BY first, second OFFSET %s LIMIT %s", (start_index, end_index - start_index))
        
        # Fetch the results
        results = cur.fetchall()

        # Print the results
#         for row in results:
#             print("Row:", type(row))

        return results

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

# Example usage: Read rows from index 1000 to 2000
res = select_rows_with_range(100, 200)
print(len(res))

100


In [2]:
def find_common_count(arr1, arr2):
    size1, size2 = len(arr1), len(arr2)
    i, j, count = 0, 0, 0

    while i < size1 and j < size2:
        if arr1[i] == arr2[j]:
            # Found an element in the intersection
            count += 1
            i += 1
            j += 1
        elif arr1[i] < arr2[j]:
            # Move the pointer in the first array
            i += 1
        else:
            # Move the pointer in the second array
            j += 1

    return count

In [3]:
def process_cocitation_row(row):
    first_in = row[1]
    
    second_in = row[3]

    first_in_nodes = first_in.split(',')
    second_in_nodes = second_in.split(',')

    first_in_nodes.sort()
    second_in_nodes.sort()

    intersection_count = find_common_count(first_in_nodes, second_in_nodes)
    
    jaccard = 0.0
    
    if len(first_in_nodes) + len(second_in_nodes) == 0:
        jaccard = 0.0
        
    else:
        jaccard = float(intersection_count/(len(first_in_nodes) + len(second_in_nodes) - intersection_count))

    return jaccard, intersection_count

for row in res:
    print(process_cocitation_row(row))

(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0026690391459074734, 3)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0031308703819661866, 10)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0009517766497461929, 9)
(1.2624349845982932e-05, 1)
(0.0, 0)
(6.565988181221274e-05, 3)
(0.02981651376146789, 13)
(0.01, 1)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(5.5008526321579844e-05, 1)
(0.0, 0)
(0.0, 0)
(0.010526315789473684, 2)
(0.0, 0)
(0.0, 0)
(1.2624509222203986e-05, 1)
(0.0, 0)
(0.0, 0)
(0.016722408026755852, 5)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.04310344827586207, 5)
(0.17509025270758122, 97)
(0.0, 0)
(0.0, 0)
(0.005136986301369863, 3)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.01680672268907563, 6)
(0.0, 0)
(0.00010595465140919687, 1)
(0.0, 0)
(0.0, 0)
(0.0024875621890547263, 1)
(0.0, 0)
(0.0, 0)
(0.0009715025906735751, 3)
(0.0, 0)
(0.004405286343612335, 1)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0013774104683195593, 2)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.0, 0)
(0.00017041581

In [13]:
def process_batch(rows):
    results = []
    for row in rows:
        first_node = row[0]
        second_node = row[2]
        
        jaccard, intersection_count = process_cocitation_row(row)
        results.append((first_node, second_node, jaccard, intersection_count))
    
    return results     

In [5]:
# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"


# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

def insert_values_into_table(values_list):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")
        cur = conn.cursor()

        
        args = ','.join(cur.mogrify("(%s, %s, %s, %s)", i).decode('utf-8') for i in values_list)
                
        cur.execute("INSERT INTO hm31.cocitation_edge_weights_cert VALUES " + (args))

        conn.commit()
        print('success')

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

In [6]:
def get_table_count():
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")  # Replace with your connection parameters
        cur = conn.cursor()

        # Execute the SELECT COUNT(*) query
        cur.execute("SELECT COUNT(*) FROM hm31.in_edges_features_cert")
        
        # Fetch the result
        result = cur.fetchone()

        # Print the result
        return result[0]

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()
count = int(get_table_count())
print(count)

44509707


In [ ]:
def insert_wrapper(start_index, end_index):
    rows = select_rows_with_range(start_index, end_index)
    results = process_batch(rows)
    insert_values_into_table(results)
    

In [20]:
from tqdm import tqdm

SPLIT_LENGTH = 1000
CHUNCKS = count // SPLIT_LENGTH

for i in tqdm(range(CHUNCKS)):
    print('a')
    rows = select_rows_with_range(i*SPLIT_LENGTH, min((i+1)* SPLIT_LENGTH, count))
    print('b')
    results = process_batch(rows)
    print('c')
    insert_values_into_table(results)
    print('d')
    print()

    if i == 10:
        break
    
    


  0%|          | 0/44509 [00:00<?, ?it/s]

a
b


  0%|          | 1/44509 [00:00<7:18:53,  1.69it/s]

c
Error: duplicate key value violates unique constraint "idx_unique_first_second"
DETAIL:  Key (first, second)=(0, 3803148) already exists.

d

a
b


  0%|          | 2/44509 [00:01<6:00:18,  2.06it/s]

...

c
Error: duplicate key value violates unique constraint "idx_unique_first_second"
DETAIL:  Key (first, second)=(1207, 147929) already exists.

d

a
b


  0%|          | 10/44509 [00:05<6:55:32,  1.78it/s]

c
Error: duplicate key value violates unique constraint "idx_unique_first_second"
DETAIL:  Key (first, second)=(1373, 3417048) already exists.

d

a
b


  0%|          | 10/44509 [00:05<7:08:41,  1.73it/s]

c
Error: duplicate key value violates unique constraint "idx_unique_first_second"
DETAIL:  Key (first, second)=(1599, 5916) already exists.

d



In [ ]:
from tqdm import tqdm



for i in tqdm(range(44500000, count)):
    rows = select_rows_with_range(i,i+1)
    results = process_batch(rows)
    insert_values_into_table(results)

In [ ]:
import multiprocessing

lst = []

SPLIT_LENGTH = 1000
CHUNCKS = count // SPLIT_LENGTH

for i in range(CHUNCKS):
    lst.append((i*SPLIT_LENGTH, min((i+1)* SPLIT_LENGTH, count)))
    
with multiprocessing.Pool(processes=50) as pool:
    results = pool.starmap(insert_wrapper, lst)